# Recommender Engine with PyTorch

In [2]:
print("hello world")

hello world


In [3]:
# Data Citation:
# F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on 
# Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. <https://doi.org/10.1145/2827872>

# ! curl http://files.grouplens.org/datasets/movielens/ml-latest-small.zip -o ml-latest-small.zip

In [4]:
# import zipfile
# with zipfile.ZipFile('ml-latest-small.zip', 'r') as zip_ref:
#     zip_ref.extractall('data')

In [5]:
# import the dataset
import pandas as pd
movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')

In [6]:
print('The dimensions of movies dataframe are:', movies_df.shape,'\nThe dimensions of ratings dataframe are:', ratings_df.shape)

The dimensions of movies dataframe are: (9742, 3) 
The dimensions of ratings dataframe are: (100836, 4)


In [7]:
# Take a look at movies_df
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
# Take a look at ratings_df
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [9]:
# Movie ID to movie name mapping
movie_names = movies_df.set_index('movieId')['title'].to_dict()
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())
print("Number of unique users:", n_users)
print("Number of unique movies:", n_items)
print("The full rating matrix will have:", n_users*n_items, 'elements.')
print('----------')
print("Number of ratings:", len(ratings_df))
print("Therefore: ", len(ratings_df) / (n_users*n_items) * 100, '% of the matrix is filled.')
print("We have an incredibly sparse matrix to work with here.")
print("And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2")
print("You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.")
print("One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data")

Number of unique users: 610
Number of unique movies: 9724
The full rating matrix will have: 5931640 elements.
----------
Number of ratings: 100836
Therefore:  1.6999683055613624 % of the matrix is filled.
We have an incredibly sparse matrix to work with here.
And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2
You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.
One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data


In [10]:
import torch
import numpy as np
from torch.autograd import Variable
# from tqdm import tqdm_notebook as tqdm
from tqdm.auto import tqdm

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # create user embeddings
        self.user_factors = torch.nn.Embedding(n_users, n_factors) # think of this as a lookup table for the input.
        # create item embeddings
        self.item_factors = torch.nn.Embedding(n_items, n_factors) # think of this as a lookup table for the input.
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)
        
    def forward(self, data):
        # matrix multiplication
        users, items = data[:,0], data[:,1]
        return (self.user_factors(users)*self.item_factors(items)).sum(1)
    # def forward(self, user, item):
    # 	# matrix multiplication
    #     return (self.user_factors(user)*self.item_factors(item)).sum(1)
    
    def predict(self, user, item):
        return self.forward(user, item)

/home/jb/.pyenv/versions/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# Creating the dataloader (necessary for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness

# Note: This isn't 'good' practice, in a MLops sense but we'll roll with this since the data is already loaded in memory.
class Loader(Dataset):
    def __init__(self):
        self.ratings = ratings_df.copy()
        
        # Extract all user IDs and movie IDs
        users = ratings_df.userId.unique()
        movies = ratings_df.movieId.unique()
        
        #--- Producing new continuous IDs for users and movies ---
        
        # Unique values : index
        self.userid2idx = {o:i for i,o in enumerate(users)}
        self.movieid2idx = {o:i for i,o in enumerate(movies)}
        
        # Obtained continuous ID for users and movies
        self.idx2userid = {i:o for o,i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}
        
        # return the id from the indexed values as noted in the lambda function down below.
        self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])
        
        
        self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
        self.y = self.ratings['rating'].values
        self.x, self.y = torch.tensor(self.x), torch.tensor(self.y) # Transforms the data to tensors (ready for torch models.)

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)

In [13]:
# num_epochs = 128
# cuda = torch.cuda.is_available()
num_epochs = 25
cuda = False

print("Is running on GPU:", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
# GPU enable if you have a GPU...
if cuda:
    model = model.cuda()

# MSE loss
loss_fn = torch.nn.MSELoss()

# ADAM optimizier
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train data
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle=True)

Is running on GPU: False
MatrixFactorization(
  (user_factors): Embedding(610, 8)
  (item_factors): Embedding(9724, 8)
)
user_factors.weight tensor([[0.0428, 0.0268, 0.0330,  ..., 0.0151, 0.0334, 0.0106],
        [0.0294, 0.0341, 0.0460,  ..., 0.0303, 0.0438, 0.0119],
        [0.0075, 0.0431, 0.0106,  ..., 0.0108, 0.0365, 0.0110],
        ...,
        [0.0173, 0.0100, 0.0228,  ..., 0.0458, 0.0119, 0.0280],
        [0.0444, 0.0149, 0.0333,  ..., 0.0463, 0.0102, 0.0064],
        [0.0428, 0.0434, 0.0107,  ..., 0.0262, 0.0187, 0.0074]])
item_factors.weight tensor([[0.0258, 0.0321, 0.0448,  ..., 0.0381, 0.0103, 0.0263],
        [0.0180, 0.0314, 0.0259,  ..., 0.0058, 0.0464, 0.0389],
        [0.0452, 0.0192, 0.0444,  ..., 0.0110, 0.0102, 0.0066],
        ...,
        [0.0034, 0.0347, 0.0188,  ..., 0.0019, 0.0018, 0.0038],
        [0.0473, 0.0427, 0.0008,  ..., 0.0253, 0.0269, 0.0318],
        [0.0232, 0.0121, 0.0484,  ..., 0.0037, 0.0217, 0.0384]])


In [14]:
# for it in tqdm(range(num_epochs)):
#     losses = []
#     for x, y in train_loader:
#          if cuda:
#             x, y = x.cuda(), y.cuda()
#             optimizer.zero_grad()
#             outputs = model(x)
#             loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
#             losses.append(loss.item())
#             loss.backward()
#             optimizer.step()
#     print("iter #{}".format(it), "Loss:", sum(losses) / len(losses))

for it in tqdm(range(num_epochs)):
    losses = []
    for x, y in train_loader:
        if cuda:
            x, y = x.cuda(), y.cuda()
        optimizer.zero_grad()
        outputs = model(x)
        loss = loss_fn(outputs.squeeze(), y.float())
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
    print(f"iter #{it} Loss: {sum(losses)/len(losses):.4f}")


  0%|          | 0/25 [00:00<?, ?it/s]

  4%|▍         | 1/25 [00:41<16:38, 41.60s/it]

iter #0 Loss: 11.0620


  8%|▊         | 2/25 [01:03<11:24, 29.75s/it]

iter #1 Loss: 4.7463


...

iter #22 Loss: 0.6579


 96%|█████████▌| 24/25 [12:49<00:27, 27.05s/it]

iter #23 Loss: 0.6570


100%|██████████| 25/25 [13:09<00:00, 31.57s/it]

iter #24 Loss: 0.6563


In [15]:
# By training the model, we will have tuned latent factors for movies and users.
c = 0
uw = 0
iw = 0 
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        if c == 0:
          uw = param.data
          c +=1
        else:
          iw = param.data
        #print('param_data', param_data)

user_factors.weight tensor([[1.4145, 1.3358, 1.3360,  ..., 1.3445, 1.3211, 1.2992],
        [1.1091, 1.1401, 1.1312,  ..., 1.0894, 1.1001, 1.0945],
        [0.7131, 0.7472, 0.6797,  ..., 0.7318, 0.7861, 0.7159],
        ...,
        [0.9580, 1.0320, 1.1253,  ..., 1.1338, 0.9735, 1.0576],
        [1.0130, 0.9789, 0.9884,  ..., 1.0306, 1.0025, 1.0058],
        [1.1990, 1.2257, 1.0900,  ..., 1.1690, 1.1126, 1.1602]])
item_factors.weight tensor([[0.4547, 0.4400, 0.4715,  ..., 0.4510, 0.4379, 0.4271],
        [0.3921, 0.3878, 0.3550,  ..., 0.3461, 0.4029, 0.3875],
        [0.4536, 0.4446, 0.4735,  ..., 0.4238, 0.3985, 0.4178],
        ...,
        [0.3217, 0.3532, 0.3373,  ..., 0.3205, 0.3200, 0.3227],
        [0.3962, 0.3918, 0.3496,  ..., 0.3744, 0.3755, 0.3812],
        [0.3839, 0.3730, 0.4091,  ..., 0.3646, 0.3822, 0.3996]])


In [16]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()

In [17]:
len(trained_movie_embeddings) # unique movie factor weights

9724

In [18]:
from sklearn.cluster import KMeans
# Fit the clusters based on the movie weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)

In [19]:
'''It can be seen here that the movies that are in the same cluster tend to have
similar genres. Also note that the algorithm is unfamiliar with the movie name
and only obtained the relationships by looking at the numbers representing how
users have responded to the movie selections.'''
for cluster in range(10):
    print("Cluster #{}".format(cluster))
    movs = []
    # Find movie indices belonging to the current cluster
    for movidx in np.where(kmeans.labels_ == cluster)[0]:
        movid = train_set.idx2movieid[movidx]
        # Check how many ratings this movie has
        rat_count = len(ratings_df.loc[ratings_df['movieId'] == movid])
        movs.append((movie_names[movid], rat_count))
    # Sort movies by rating count in descending order and print top 10
    for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
        print("\t", mov[0])

Cluster #0
	 Waterworld (1995)
	 Ace Ventura: When Nature Calls (1995)
	 Nutty Professor, The (1996)
	 Mission: Impossible II (2000)
	 Charlie's Angels (2000)
	 Honey, I Shrunk the Kids (1989)
	 Lost World: Jurassic Park, The (1997)
	 Austin Powers in Goldmember (2002)
	 Blair Witch Project, The (1999)
	 Casper (1995)
Cluster #1
	 Pulp Fiction (1994)
	 Silence of the Lambs, The (1991)
	 Star Wars: Episode IV - A New Hope (1977)
	 Braveheart (1995)
	 Schindler's List (1993)
	 Toy Story (1995)
	 American Beauty (1999)
	 Seven (a.k.a. Se7en) (1995)
	 Lord of the Rings: The Fellowship of the Ring, The (2001)
	 Star Wars: Episode VI - Return of the Jedi (1983)
Cluster #2
	 Jason X (2002)
	 When a Stranger Calls (2006)
	 Police Academy: Mission to Moscow (1994)
	 House Party 2 (1991)
	 Amityville II: The Possession (1982)
	 Epic Movie (2007)
	 Wicker Man, The (2006)
	 House of the Dead, The (2003)
	 Book of Shadows: Blair Witch 2 (2000)
	 Drowning Mona (2000)
Cluster #3
	 Ace Ventura: Pet De